### Ollama

In [1]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler 

`CallbackManager` helps manage multiple callback handlers. It allows you to specify what should happen when certain events occur

`StreamingStdOutCallbackHandler` is a specific callback handler that prints output to the standard output as the model generates text.

In [7]:
llm = Ollama(
    model="gemma2:2b", 
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
)



llm("Explain artificial intelligence in short")

Artificial intelligence (AI) is the ability of machines to think and act like humans.  

**Think of it as:**

* **Teaching computers to learn from data**: Just like you learned by experience, AI systems use massive amounts of information to improve their skills over time. 
* **Solving problems**:  AI can analyze information, identify patterns, and make decisions based on what it learns. This helps solve real-world issues in areas like healthcare, transportation, and finance.
* **Improving our lives**: AI makes things easier and more efficient for us - think self-driving cars, personalized recommendations on Netflix, or virtual assistants that answer your questions.


**Essentially, AI aims to mimic human intelligence in machines, allowing them to perform tasks that typically require human intelligence.** 


'Artificial intelligence (AI) is the ability of machines to think and act like humans.  \n\n**Think of it as:**\n\n* **Teaching computers to learn from data**: Just like you learned by experience, AI systems use massive amounts of information to improve their skills over time. \n* **Solving problems**:  AI can analyze information, identify patterns, and make decisions based on what it learns. This helps solve real-world issues in areas like healthcare, transportation, and finance.\n* **Improving our lives**: AI makes things easier and more efficient for us - think self-driving cars, personalized recommendations on Netflix, or virtual assistants that answer your questions.\n\n\n**Essentially, AI aims to mimic human intelligence in machines, allowing them to perform tasks that typically require human intelligence.** \n'

### Chaining

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [8]:
chain_llm = Ollama(
    model="gemma2:2b",
    temperature=0.9,
)


In [10]:
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Give me a short summary of {topic}",
)

chain = LLMChain(
    llm=chain_llm,
    prompt=prompt,
    verbose=True,
)

/home/jerlshin/ENV/env_torch/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
chain.run("Artificial Intelligence")



> Entering new LLMChain chain...
Prompt after formatting:
Give me a short summary of Artificial Intelligence

> Finished chain.


'Artificial intelligence (AI) is the ability of computer systems to mimic human intelligence, such as learning, problem-solving, and decision making. \n\n**Here\'s a breakdown:**\n\n* **Goal:** To create machines that can perform tasks typically requiring human intelligence.\n* **How it works:** AI uses algorithms, data sets, and computational power to learn from experiences and make predictions or decisions. \n* **Types of AI:** There are different types:\n    * **Machine Learning:**  Algorithms that "learn" from data without explicit programming.\n    * **Deep Learning:** A subset of machine learning using artificial neural networks inspired by the human brain.\n    * **Natural Language Processing (NLP):** Allows computers to understand, interpret, and generate human language.\n    * **Computer Vision:** Enables computers to "see" and analyze images or videos.\n\n**Applications:** AI is revolutionizing many industries, including:\n\n* **Healthcare:** Diagnosing diseases, developing p

### Retrieval Augmented Generation

In [2]:
import argparse
import sys

from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings import OllamaEmbeddings

from langchain import hub

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
url = 'http://example.com'

print(f"using URL: {url}")


using URL: http://example.com


In [4]:
loader = WebBaseLoader(url) # langchain.document_loaders.WebBaseLoader
data = loader.load()

In [ ]:
ollama_model = "gemma2:2b"

# Split into chunks 
# split large documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

print(f"Split into {len(all_splits)} chunks")

# creates a vectorstore to store and retrieve documents embeddings efficiently
vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OllamaEmbeddings(model=ollama_model))

Split into 1 chunks


In [6]:
all_splits

[Document(metadata={'source': 'http://example.com', 'title': 'Example Domain', 'language': 'No language found.'}, page_content='Example Domain\n\n\n\n\n\n\n\nExample Domain\nThis domain is for use in illustrative examples in documents. You may use this\n    domain in literature without prior coordination or asking for permission.\nMore information...')]

In [52]:
# Retrieve
# question = "What are the latest headlines on {url}?"
# docs = vectorstore.similarity_search(question)
print(f"Loaded {len(data)} documents")
# print(f"Retrieved {len(docs)} documents")


Loaded 1 documents


In [53]:
import pprint
import json
import yaml

In [54]:
# retrieves a predefined prompt template from Langchain Hub
QA_CHAIN_PROMPT = hub.pull("rlm/rag-prompt-llama")

/home/jerlshin/ENV/env_torch/lib/python3.12/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [55]:
print(type(QA_CHAIN_PROMPT))

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>


In [56]:
print(yaml.dump(QA_CHAIN_PROMPT))

!!python/object:langchain_core.prompts.chat.ChatPromptTemplate
__dict__:
  input_types: {}
  input_variables:
  - context
  - question
  messages:
  - !!python/object:langchain_core.prompts.chat.HumanMessagePromptTemplate
    __dict__:
      additional_kwargs: {}
      prompt: !!python/object:langchain_core.prompts.prompt.PromptTemplate
        __dict__:
          input_types: {}
          input_variables:
          - context
          - question
          metadata: null
          name: null
          optional_variables: []
          output_parser: null
          partial_variables: {}
          tags: null
          template: "[INST]<<SYS>> You are an assistant for question-answering tasks.\
            \ Use the following pieces of retrieved context to answer the question.\
            \ If you don't know the answer, just say that you don't know. Use three\
            \ sentences maximum and keep the answer concise.<</SYS>> \nQuestion: {question}\
            \ \nContext: {context} \n

In [57]:
# LLM
llm = Ollama(model=ollama_model,
            verbose=True,
            callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

print(f"Loaded LLM model {llm.model}")

Loaded LLM model gemma2:2b


/home/jerlshin/ENV/env_torch/lib/python3.12/site-packages/langchain_core/language_models/llms.py:309: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  warnings.warn(


In [31]:
llm.model

'gemma2:2b'

In [36]:
output = llm("What is the capital of India?")

/home/jerlshin/ENV/env_torch/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


The capital of India is **New Delhi**. 🇮🇳 


In [40]:
from langchain.callbacks import FileCallbackHandler
from langchain.callbacks import StreamlitCallbackHandler

from langchain.callbacks.base import BaseCallbackHandler


class CustomCallbackHandler(BaseCallbackHandler):
    def on_text(self, text):
        print(f"Custom Output: {text}")
    
callback_manager = CallbackManager([CustomCallbackHandler()])

In [58]:
# QA chain
from langchain.chains import RetrievalQA

# retrieval chain Qa
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)
# Ask a question
question = f"What does this article talk about?"
result = qa_chain({"query": question})

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


This article discusses the purpose and usage of a domain named "Example Domain." It provides instructions on how to utilize the domain, even without prior authorization from its creators.  It is intended for illustrative purposes in writing.  [/INST] 


In [60]:
len(vectorstore)

1